### Introdução Camada Gold: 

A camada Gold tem como objetivo organizar os dados em um formato analítico otimizado para consumo por ferramentas de Business Intelligence e análises exploratórias.

Nesta etapa, os dados tratados na camada Silver são modelados segundo os princípios de Data Warehouse, utilizando uma abordagem de modelo dimensional (esquema estrela), no qual uma tabela fato centraliza as métricas do negócio e se relaciona com tabelas dimensão que descrevem os contextos dessas métricas.

Essa modelagem facilita consultas analíticas, melhora a performance de agregações e garante maior clareza semântica para análises e dashboards.

In [0]:
df = spark.table("workspace.games_analytics_silver.games_march_2025")


### Criação das Dimensões: 

Nesta etapa, foram criadas as tabelas dimensão, responsáveis por armazenar atributos descritivos que contextualizam os dados da tabela fato.

Cada dimensão possui uma chave substituta (ID), utilizada para garantir relacionamentos consistentes e evitar dependência direta de campos textuais na tabela fato.

In [0]:
from pyspark.sql import functions as F

dim_publisher = (
    df.select("publisher_normalized")
      .distinct()
      .withColumnRenamed("publisher_normalized", "publisher_name")
      .withColumn("publisher_id", F.monotonically_increasing_id())
)

dim_publisher.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_publisher")

A dimensão da publisher foi criada a partir do tratamento e modelagem da coluna da publisher na camada silver, trazendo uma centralização e padronização dos dados.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_publisher
LIMIT (10)

publisher_name,publisher_id
Atari,0
Dinosaur Polo Club,1
Rose City Games,2
Midnight Hub,3
Microprose Software,4
Ed Del Castillo,5
Hanako Games,6
Conglomerate 5,7
Sadsquare Studio,8
Rival Games Ltd,9


In [0]:
from pyspark.sql import functions as F

dim_developer = (
    df.select("developer_normalized")
      .distinct()
      .withColumnRenamed("developer_normalized", "developer_name")
      .withColumn("developer_id", F.monotonically_increasing_id())
)

dim_developer.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_developer")

A dimensão developer foi criada a partir da padronização da mesma coluna na camada silver, evitando valores duplicados e centralizando os valores.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_developer
LIMIT (10)

developer_name,developer_id
Sledgehammer Games,0
Dinosaur Polo Club,1
Rose City Games,2
Innerspacevr,3
Midnight Hub,4
Microprose Software,5
Ed Del Castillo,6
Darkpaw Games,7
Hanako Games,8
Mobius Digital,9


In [0]:
from pyspark.sql import functions as F


dim_genre = (
    df.select(F.col("genre_1").alias("genre"))
      .union(df.select(F.col("genre_2").alias("genre")))
      .where(F.col("genre").isNotNull())
      .distinct()
      .withColumn("genre_id", F.monotonically_increasing_id())
)

dim_genre.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_genre")


A dimensão de gênero consolida os gêneros tratados na camada Silver, permitindo análises categóricas e evitando duplicidade de valores textuais na tabela fato.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_genre
LIMIT (10)

genre,genre_id
indie,0
'violent',1
'action',2
'racing',3
,4
'free to play',5
'rpg',6
'strategy',7
'casual',8
'sports',9


In [0]:
from pyspark.sql import functions as F

dim_time = (
    df.select("release_date")
      .distinct()
      .withColumn("time_id", F.monotonically_increasing_id())
      .withColumn("release_year", F.year("release_date"))
      .withColumn("release_month", F.month("release_date"))
      .withColumn("release_month_name", F.date_format("release_date", "MMMM"))
      .withColumn("release_quarter", F.quarter("release_date"))
      .withColumn("release_semester",
          F.when(F.month("release_date") <= 6, 1).otherwise(2)
      )
)

dim_time.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_time")


A dimensão de tempo foi criada a partir da data de lançamento dos jogos, permitindo análises por ano, mês e outros recortes temporais.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_time
LIMIT (10)

release_date,time_id,release_year,release_month,release_month_name,release_quarter,release_semester
2016-12-14,0,2016,12,December,4,2
2020-12-25,1,2020,12,December,4,2
2012-08-24,2,2012,8,August,3,2
2023-04-19,3,2023,4,April,2,1
2017-01-27,4,2017,1,January,1,1
2005-08-16,5,2005,8,August,3,2
2020-05-14,6,2020,5,May,2,1
2010-06-07,7,2010,6,June,2,1
2014-08-21,8,2014,8,August,3,2
2012-10-31,9,2012,10,October,4,2


In [0]:
from pyspark.sql import functions as F

dim_jogo = (
    df.select(
        F.col("name").alias("name")
    )
    .dropDuplicates()
    .withColumn("game_id", F.monotonically_increasing_id())
    .withColumn("created_at", F.current_timestamp())
)

dim_jogo.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_jogo")

A dimensão do jogo foi criada a partir do tratamento e agrupamento dos jogos por nome na camada silver, evitando duplicatas.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_jogo
LIMIT (10)

name,game_id,created_at
DOOM VFR,0,2025-12-19T15:10:21.479Z
The Wanderer: Frankenstein’s Creature,1,2025-12-19T15:10:21.479Z
FEZ,2,2025-12-19T15:10:21.479Z
Lumini,3,2025-12-19T15:10:21.479Z
Layers of Fear (2016),4,2025-12-19T15:10:21.479Z
Long Live The Queen,5,2025-12-19T15:10:21.479Z
"Warhammer 40,000: Sanctus Reach",6,2025-12-19T15:10:21.479Z
I fell from Grace,7,2025-12-19T15:10:21.479Z
Beyond Blue,8,2025-12-19T15:10:21.479Z
NOBUNAGA'S AMBITION: Sphere of Influence,9,2025-12-19T15:10:21.479Z


In [0]:
fact = (
    df
    # DIM JOGO
    .join(
        dim_jogo,
        df.name == dim_jogo.name,
        "left"
    )

    # DIM PUBLISHER
    .join(
        dim_publisher,
        df.publisher_normalized == dim_publisher.publisher_name,
        "left"
    )

    # DIM DEVELOPER
    .join(
        dim_developer,
        df.developer_normalized == dim_developer.developer_name,
        "left"
    )

    # DIM GENRE (duas vezes)
    .join(
        dim_genre.alias("g1"),
        df.genre_1 == F.col("g1.genre"),
        "left"
    )
    .join(
        dim_genre.alias("g2"),
        df.genre_2 == F.col("g2.genre"),
        "left"
    )

    # DIM TEMPO
    .join(
        dim_time,
        "release_date",
        "left"
    )
)


A tabela fato centraliza as métricas quantitativas relacionadas aos jogos, como avaliações, preços e indicadores derivados, além de armazenar as chaves estrangeiras que referenciam as dimensões criadas anteriormente.

Os relacionamentos foram realizados por meio de joins entre os dados da camada Silver e as dimensões, substituindo atributos textuais por seus respectivos identificadores.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.fact_games
LIMIT (10)

game_id,publisher_id,developer_id,genre_1_id,genre_2_id,time_id,price,approval_pct,metacritic_score,positive_ratings_total,negative_ratings_total,is_indie,total_reviews,windows,mac,linux,approval_weighted,review_confidence
1,703,1425,10,8,1054,15.99,84.95,69.0,237,42,true,279,true,true,false,80.62,Baixa
12,477,682,10,10,162,14.99,57.64,73.0,332,244,true,576,true,true,true,78.04,Baixa
15,285,1008,2,10,1561,9.99,83.09,58.0,231,47,true,278,true,false,false,80.52,Baixa
32,154,214,10,10,1761,5.99,98.29,78.0,345,6,true,351,true,true,true,81.54,Baixa
33,1190,1781,6,11,2019,14.99,73.53,78.0,100,36,false,136,true,true,false,80.2,Baixa
39,786,1171,6,6,434,9.99,68.74,64.0,464,211,true,675,true,false,false,79.0,Baixa
52,1356,2025,2,2,781,9.99,79.35,69.0,392,102,true,494,true,false,false,80.29,Baixa
56,761,1127,1,9,531,11.99,67.44,48.0,174,84,false,258,true,false,false,79.75,Baixa
57,876,986,2,3,1352,4.99,85.93,58.0,116,19,false,135,true,false,false,80.52,Baixa
58,770,714,10,10,363,2.99,70.12,63.0,115,49,false,164,true,false,false,80.05,Baixa


In [0]:
fact_games = fact.select(
    F.col("game_id"),
    F.col("publisher_id"),
    F.col("developer_id"),
    F.col("g1.genre_id").alias("genre_1_id"),
    F.col("g2.genre_id").alias("genre_2_id"),
    F.col("time_id"),

    # métricas
    F.col("price"),
    F.col("approval_pct"),
    F.col("metacritic_score"),
    F.col("positive_ratings_total"),
    F.col("negative_ratings_total"),
    F.col("is_indie"),
    F.col("total_reviews"),
    F.col("windows"),
    F.col("mac"),
    F.col("linux"),
    F.col("approval_weighted"),
    F.col("review_confidence")
)



Após a criação da Fato e o ajuste no relacionamento dela com as tabelas DIM, a partir dos valores vindos na tabela tratada da camada silver, é necessário carregar as métricas de valores diretamente na FATO.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.fact_games
LIMIT (10)

game_id,publisher_id,developer_id,genre_1_id,genre_2_id,time_id,price,approval_pct,metacritic_score,positive_ratings_total,negative_ratings_total,is_indie,total_reviews,windows,mac,linux,approval_weighted,review_confidence
1,703,1425,10,8,1054,15.99,84.95,69.0,237,42,true,279,true,true,false,80.62,Baixa
12,477,682,10,10,162,14.99,57.64,73.0,332,244,true,576,true,true,true,78.04,Baixa
15,285,1008,2,10,1561,9.99,83.09,58.0,231,47,true,278,true,false,false,80.52,Baixa
32,154,214,10,10,1761,5.99,98.29,78.0,345,6,true,351,true,true,true,81.54,Baixa
33,1190,1781,6,11,2019,14.99,73.53,78.0,100,36,false,136,true,true,false,80.2,Baixa
39,786,1171,6,6,434,9.99,68.74,64.0,464,211,true,675,true,false,false,79.0,Baixa
52,1356,2025,2,2,781,9.99,79.35,69.0,392,102,true,494,true,false,false,80.29,Baixa
56,761,1127,1,9,531,11.99,67.44,48.0,174,84,false,258,true,false,false,79.75,Baixa
57,876,986,2,3,1352,4.99,85.93,58.0,116,19,false,135,true,false,false,80.52,Baixa
58,770,714,10,10,363,2.99,70.12,63.0,115,49,false,164,true,false,false,80.05,Baixa


In [0]:
fact_games.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.fact_games")


Ao final do processo, as tabelas dimensão e a tabela fato foram persistidas na camada Gold, tornando os dados prontos para consumo analítico, criação de views e integração com ferramentas de BI.